In [1]:

import pandas as pd
import numpy as np
import tensorflow as tf
import time

from nltk.tokenize import sent_tokenize, word_tokenize
from nltk.corpus import stopwords
import gensim
# from collections import Counter
# from functools import partial
# from tqdm import tqdm_notebook

/Library/Frameworks/Python.framework/Versions/3.6/lib/python3.6/importlib/_bootstrap.py:219: RuntimeWarning: compiletime version 3.5 of module 'tensorflow.python.framework.fast_tensor_util' does not match runtime version 3.6
  return f(*args, **kwds)


In [3]:
df=pd.read_csv('testdata.csv')
print(df.head(50))

    Unnamed: 0   Id                                               Name  \
0            0   c0  University of Western Australia v Gray (No 21)...   
1            1   c1  University of Western Australia v Gray (No 21)...   
2            2   c2  University of Western Australia v Gray (No 21)...   
3            3   c3  University of Western Australia v Gray (No 21)...   
4            4   c4  University of Western Australia v Gray (No 21)...   
5            5   c5  University of Western Australia v Gray (No 21)...   
6            6   c6  University of Western Australia v Gray (No 21)...   
7            7   s0  University of Western Australia v Gray (No 21)...   
8            8   s1  University of Western Australia v Gray (No 21)...   
9            9   s2  University of Western Australia v Gray (No 21)...   
10          10   s3  University of Western Australia v Gray (No 21)...   
11          11   s4  University of Western Australia v Gray (No 21)...   
12          12   s5  University of Wes

In [10]:
df[(df.file_id=='08_1056.xml') & (df.Id=='s19')]['text']

26    The UWA proceedings against Sirtex would be di...
Name: text, dtype: object

In [2]:
window_size=2;

def read_data(file_name):
    with open(file_name,'r') as f:
        word_vocab = set()  # not using list to avoid duplicate entry
        word_count={};
        
        word2vector = {}
        for line in f:
            line_ = line.strip() #Remove white space
            words_Vec = line_.split()
            word_vocab.add(words_Vec[0])
            if words_Vec[0] in word_count:
                print("Already exists",words_Vec[0])
                word_count[words_Vec[0]]=word_count.get(words_Vec[0])+1;
            else:
                word_count[words_Vec[0]]=1;
                
            word2vector[words_Vec[0]] = np.array(words_Vec[1:],dtype=float)
    print("Total Words in DataSet:",len(word_vocab))
    return word_vocab,word2vector,word_count

vocab, w2v,cnt = read_data("glove_6B_200d.txt");
print(str(vocab)[1:100])




Total Words in DataSet: 400000
'126.40', 'pombo', 'grito', 'olpc', 'luas', '38.3', 'zizic', 'koepp', 'scriptores', '1830-31', '197


In [3]:
print(cnt['the'])

1


In [4]:
word_count={};


In [5]:
#print(w2v['DKW'])
try:
    c = w2v['DKW']
except KeyError:
    print("not in vocabulary")
    c = 0

not in vocabulary


In [6]:
full_data=pd.read_csv('./data.csv');
print(np.shape(full_data))


(866842, 5)


In [7]:
data=full_data[:50000]


# for index, row in data.iterrows():
#     text_tokens=word_tokenize(row['text']);
#     for token in text_tokens:
#         if token in word_count:
#             print("Already exists",token)
#             word_count[token]=word_count.get(token)+1;
#         else:
#             word_count[token]=1;
            
# print(word_count['the'])
# print(data.head(5));
# data.to_csv('sample_data.csv');

In [8]:
catch_phrases=full_data[full_data['Id'].str.startswith(('c'))]
sent_phrases=full_data[full_data['Id'].str.startswith(('s'))]
print(len(catch_phrases))
print(len(sent_phrases))
less_text=catch_phrases[catch_phrases['text'].str.len() <10]
more_text=catch_phrases[catch_phrases['text'].str.len() >= 20]
print(len(less_text))
print(len(more_text))

31075
835767
2526
23574


In [9]:
# for index, row in data.iterrows()[:10]:
#     print ( row['file_id'],row['Id'],row['text'],)

In [ ]:
def get_word_count(string):
    tokens = string.split()
    n_tokens = len(tokens)
    return n_tokens  

def ReLU(x):
    return abs(x) * (x > 0)

In [ ]:
def get_features(inputs):
    out=np.zeros((len(inputs),200));
    for i in range(len(inputs)):
        try:
            c = w2v[inputs[i]]
        except KeyError:
            c = np.zeros((1,200));
        out[i]=c;
    
    return np.transpose(out);
        

In [ ]:
def sent2vector(sent):
    words = word_tokenize(sent.lower())
    
    # Here we weight-average each word in sentence by 1/log(count[word])
    emb = [w2v[w] for w in words if w in w2v]
    weights = [1./word_count[w] for w in words if w in w2v]
    
    if len(emb) == 0:
        return np.zeros(200, dtype=np.float32)
    else:
        return np.dot(weights, emb) / np.sum(weights)
    

In [ ]:
def get_features_row(file_id,Id,text):
    text=text.lower();
    
    word_count=get_word_count(text);
    words=text.split()
    input_filter=np.random.rand(300,200*(2*window_size+1));
    sentence_vector=np.zeros((300,1));
    count=0;
    rows_list=[]
    if word_count>2*window_size+1:
        for i in range(window_size,word_count-window_size):
            
            inputs=words[i-window_size:i+window_size+1];
            #print(inputs);
            #input_features=ReLU(get_features(inputs));
            #input_features=np.reshape(input_features,(200*(2*window_size+1),1));
            #input_features=ReLU(np.matmul(input_filter,input_features));
            #print(np.shape(input_features));
            rows_list.append({"file_id":file_id,"Id":Id,"words":inputs})
            #print(file_id,Id,inputs);
            #sentence_vector=np.add(sentence_vector,input_features);
            #count=count+1;
        #sentence_vector= np.true_divide(sentence_vector, 4);
        #print(np.shape(sentence_vector));
    #print(rows_list)
    child_dataframe=pd.DataFrame(rows_list);
    print(child_dataframe.head(5))
    return child_dataframe
        
        
    

In [ ]:
start=time.time();
words_dataframes=[];

for index, row in data.iterrows():
    words_dataframes.append(get_features_row(row['file_id'],row['Id'],row['text']))
end=time.time();

words_df=pd.concat(words_dataframes,ignore_index=True)

print("total time:",end-start);

Empty DataFrame
Columns: []
Index: []
Empty DataFrame
Columns: []
Index: []
Empty DataFrame
Columns: []
Index: []
Empty DataFrame
Columns: []
Index: []
   Id      file_id                                             words
0  c4  08_1056.xml              [variety, of, factors, relevant, to]
1  c4  08_1056.xml           [of, factors, relevant, to, assessment]
2  c4  08_1056.xml           [factors, relevant, to, assessment, of]
3  c4  08_1056.xml  [relevant, to, assessment, of, unreasonableness]
Empty DataFrame
Columns: []
Index: []
Empty DataFrame
Columns: []
Index: []
   Id      file_id                                     words
0  s0  08_1056.xml                  [1, on, 17, april, 2008]
1  s0  08_1056.xml                [on, 17, april, 2008, the]
2  s0  08_1056.xml       [17, april, 2008, the, application]
3  s0  08_1056.xml  [april, 2008, the, application, brought]
4  s0  08_1056.xml     [2008, the, application, brought, by]
   Id      file_id                           words
0  s1  08_

    Id      file_id                                  words
0  s27  08_1056.xml    [within, 30, days, of, settlement,]
1  s27  08_1056.xml    [30, days, of, settlement,, sirtex]
2  s27  08_1056.xml    [days, of, settlement,, sirtex, to]
3  s27  08_1056.xml   [of, settlement,, sirtex, to, allot]
4  s27  08_1056.xml  [settlement,, sirtex, to, allot, and]
Empty DataFrame
Columns: []
Index: []
    Id      file_id                             words
0  s29  08_1056.xml  [the, proceedings, in, wad, 292]
1  s29  08_1056.xml   [proceedings, in, wad, 292, of]
2  s29  08_1056.xml          [in, wad, 292, of, 2004]
3  s29  08_1056.xml          [wad, 292, of, 2004, in]
4  s29  08_1056.xml           [292, of, 2004, in, so]
Empty DataFrame
Columns: []
Index: []
    Id      file_id                            words
0  s31  08_1056.xml       [a, deed, of, release, to]
1  s31  08_1056.xml      [deed, of, release, to, be]
2  s31  08_1056.xml  [of, release, to, be, executed]
3  s31  08_1056.xml  [release, to,

    Id      file_id                                words
0  s61  08_1056.xml           [on, 15, march, 2007, the]
1  s61  08_1056.xml   [15, march, 2007, the, solicitors]
2  s61  08_1056.xml  [march, 2007, the, solicitors, for]
3  s61  08_1056.xml    [2007, the, solicitors, for, uwa]
4  s61  08_1056.xml   [the, solicitors, for, uwa, wrote]
    Id      file_id                             words
0  s62  08_1056.xml     [we, refer, to, your, letter]
1  s62  08_1056.xml  [refer, to, your, letter, dated]
2  s62  08_1056.xml     [to, your, letter, dated, 14]
3  s62  08_1056.xml  [your, letter, dated, 14, march]
4  s62  08_1056.xml  [letter, dated, 14, march, 2007]
Empty DataFrame
Columns: []
Index: []
    Id      file_id                                      words
0  s64  08_1056.xml           [the, offer, of, settlement, is]
1  s64  08_1056.xml  [offer, of, settlement, is, unacceptable]
2  s64  08_1056.xml     [of, settlement, is, unacceptable, to]
3  s64  08_1056.xml    [settlement, is, unac

    Id      file_id                              words
0  s96  08_1056.xml       [dr, gray, had, no, control]
1  s96  08_1056.xml     [gray, had, no, control, over]
2  s96  08_1056.xml      [had, no, control, over, the]
3  s96  08_1056.xml  [no, control, over, the, conduct]
4  s96  08_1056.xml  [control, over, the, conduct, of]
    Id      file_id                                    words
0  s97  08_1056.xml    [17, counsel, for, sirtex, submitted]
1  s97  08_1056.xml  [counsel, for, sirtex, submitted, that]
2  s97  08_1056.xml       [for, sirtex, submitted, that, it]
3  s97  08_1056.xml       [sirtex, submitted, that, it, was]
4  s97  08_1056.xml        [submitted, that, it, was, clear]
    Id      file_id                                              words
0  s98  08_1056.xml                    [18, in, its, written, outline]
1  s98  08_1056.xml                    [in, its, written, outline, of]
2  s98  08_1056.xml          [its, written, outline, of, submissions,]
3  s98  08_1056.xml 

     Id      file_id                                  words
0  s131  08_1056.xml  [professor, barber's, reply, to, the]
1  s131  08_1056.xml       [barber's, reply, to, the, gorn]
2  s131  08_1056.xml         [reply, to, the, gorn, letter]
3  s131  08_1056.xml           [to, the, gorn, letter, was]
4  s131  08_1056.xml      [the, gorn, letter, was, written]
Empty DataFrame
Columns: []
Index: []
     Id      file_id                                              words
0  s133  08_1056.xml            [uwa, adopted, a, deliberate, strategy]
1  s133  08_1056.xml             [adopted, a, deliberate, strategy, of]
2  s133  08_1056.xml         [a, deliberate, strategy, of, withholding]
3  s133  08_1056.xml  [deliberate, strategy, of, withholding, knowle...
4  s133  08_1056.xml         [strategy, of, withholding, knowledge, of]
Empty DataFrame
Columns: []
Index: []
     Id      file_id                         words
0  s135  08_1056.xml   [sirtex, was, not, put, on]
1  s135  08_1056.xml   [was, n

...

    Id     file_id                               words
0  s98  06_433.xml   [(3), where, an, application, is]
1  s98  06_433.xml  [where, an, application, is, made]
2  s98  06_433.xml     [an, application, is, made, to]
3  s98  06_433.xml    [application, is, made, to, the]
4  s98  06_433.xml          [is, made, to, the, court]
    Id     file_id                                        words
0  s99  06_433.xml            [(4), on, an, application, under]
1  s99  06_433.xml     [on, an, application, under, subsection]
2  s99  06_433.xml   [an, application, under, subsection, (1),]
3  s99  06_433.xml  [application, under, subsection, (1),, the]
4  s99  06_433.xml        [under, subsection, (1),, the, court]
     Id     file_id                           words
0  s100  06_433.xml   [(5), where, the, court, has]
1  s100  06_433.xml  [where, the, court, has, made]
2  s100  06_433.xml     [the, court, has, made, an]
3  s100  06_433.xml   [court, has, made, an, order]
4  s100  06_433.xml   [has

     Id     file_id                                        words
0  s127  06_433.xml    [this, consideration, applies, to, final]
1  s127  06_433.xml  [consideration, applies, to, final, orders]
2  s127  06_433.xml           [applies, to, final, orders, made]
3  s127  06_433.xml             [to, final, orders, made, under]
4  s127  06_433.xml            [final, orders, made, under, the]
     Id     file_id                                    words
0  s128  06_433.xml        [the, final, orders, made, under]
1  s128  06_433.xml        [final, orders, made, under, the]
2  s128  06_433.xml      [orders, made, under, the, section]
3  s128  06_433.xml         [made, under, the, section, are]
4  s128  06_433.xml  [under, the, section, are, necessarily]
     Id     file_id                                   words
0  s129  06_433.xml     [26, the, circumstances, in, which]
1  s129  06_433.xml    [the, circumstances, in, which, the]
2  s129  06_433.xml  [circumstances, in, which, the, court]
3  s

     Id     file_id                                       words
0  s159  06_433.xml  [these, considerations, are, relevant, to]
1  s159  06_433.xml    [considerations, are, relevant, to, the]
2  s159  06_433.xml     [are, relevant, to, the, admissibility]
3  s159  06_433.xml      [relevant, to, the, admissibility, of]
4  s159  06_433.xml          [to, the, admissibility, of, some]
     Id     file_id                                    words
0  s160  06_433.xml     [32, specific, objection, has, been]
1  s160  06_433.xml  [specific, objection, has, been, taken]
2  s160  06_433.xml        [objection, has, been, taken, on]
3  s160  06_433.xml           [has, been, taken, on, behalf]
4  s160  06_433.xml            [been, taken, on, behalf, of]
     Id     file_id                                 words
0  s161  06_433.xml  [it, was, submitted, that, pursuant]
1  s161  06_433.xml  [was, submitted, that, pursuant, to]
2  s161  06_433.xml    [submitted, that, pursuant, to, s]
3  s161  06_433.xm

     Id     file_id                                        words
0  s194  06_433.xml           [his, affidavit, also, refers, to]
1  s194  06_433.xml  [affidavit, also, refers, to, transactions]
2  s194  06_433.xml      [also, refers, to, transactions, which]
3  s194  06_433.xml        [refers, to, transactions, which, he]
4  s194  06_433.xml     [to, transactions, which, he, describes]
     Id     file_id                                            words
0  s195  06_433.xml  [other, transactions, involve, the, withdrawal]
1  s195  06_433.xml     [transactions, involve, the, withdrawal, of]
2  s195  06_433.xml            [involve, the, withdrawal, of, funds]
3  s195  06_433.xml                 [the, withdrawal, of, funds, in]
4  s195  06_433.xml          [withdrawal, of, funds, in, connection]
Empty DataFrame
Columns: []
Index: []
     Id     file_id                                        words
0  s197  06_433.xml  [supplementary, affidavit, of, oren, zohar]
1  s197  06_433.xml         

     Id     file_id                                              words
0  s228  06_433.xml              [he, also, exhibits, transcripts, of]
1  s228  06_433.xml    [also, exhibits, transcripts, of, examinations]
2  s228  06_433.xml  [exhibits, transcripts, of, examinations, cond...
3  s228  06_433.xml  [transcripts, of, examinations, conducted, under]
4  s228  06_433.xml            [of, examinations, conducted, under, s]
Empty DataFrame
Columns: []
Index: []
     Id     file_id                                 words
0  s230  06_433.xml   [the, second, affidavit, of, kevin]
1  s230  06_433.xml  [second, affidavit, of, kevin, chin]
2  s230  06_433.xml   [affidavit, of, kevin, chin, sworn]
3  s230  06_433.xml           [of, kevin, chin, sworn, 6]
4  s230  06_433.xml        [kevin, chin, sworn, 6, april]


In [ ]:
end=time.time();
print(words_df.head(5000))
print("total time:",end-start);

In [ ]:
words_df.to_csv('./words_data.csv');

In [ ]:
file_data=full_data[full_data.file_id=='08_1056.xml'];
print(file_data['text'][:-1])

final_str="";

for txt in file_data['text'][:-1]:
    final_str=final_str+txt;
print(final_str)
    